<a href="https://colab.research.google.com/github/ShahrzadZolghadr/Sentiment-Analysis/blob/main/SA_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Just to test
!pip install hazm
!pip install -q clean-text[gpl]
#!pip install transformers

In [ ]:
import pandas as pd
import hazm
import re
from cleantext import clean
import os
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import joblib
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!unzip drive/MyDrive/snappfood

Archive:  drive/MyDrive/snappfood.zip
replace __MACOSX/._snappfood? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._snappfood    
replace snappfood/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: snappfood/test.csv      
replace __MACOSX/snappfood/._test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/snappfood/._test.csv  
replace snappfood/dev.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: snappfood/dev.csv       
replace __MACOSX/snappfood/._dev.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/snappfood/._dev.csv  
replace snappfood/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: snappfood/train.csv     
replace __MACOSX/snappfood/._train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/snappfood/._train.csv  


In [ ]:
train = pd.read_csv('snappfood/train.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
test = pd.read_csv('snappfood/test.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
dev = pd.read_csv('snappfood/dev.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
train.head()

,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [ ]:
train = train[['comment', 'label_id']]
test = test[['comment', 'label_id']]
dev = dev[['comment', 'label_id']]
train.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


## Preprocessing text column data

In [ ]:
# calculate the length of comments based on their words
train['comment_len_by_words'] = train['comment'].apply(lambda t: len(hazm.word_tokenize(t)))
min_max_len = train["comment_len_by_words"].min(), train["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 2 	Max: 378


In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [ ]:
# cleaning comments
train['cleaned_comment'] = train['comment'].apply(cleaning)
test['cleaned_comment'] = test['comment'].apply(cleaning)
dev['cleaned_comment'] = dev['comment'].apply(cleaning)

train.head()

,comment,label_id,comment_len_by_words,cleaned_comment
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1,9,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0,28,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1,19,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0,21,عالللی بود همه چه درست و به اندازه و کیفیت خوب...
4,شیرینی وانیلی فقط یک مدل بود.,0,7,شیرینی وانیلی فقط یک مدل بود.


In [ ]:
train = train[['cleaned_comment', 'label_id']]
test = test[['cleaned_comment', 'label_id']]
dev = dev[['cleaned_comment', 'label_id']]
train.head()

,cleaned_comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [ ]:
x_train, y_train = train['cleaned_comment'].values.tolist(), train['label_id'].values.tolist()
x_test, y_test = test['cleaned_comment'].values.tolist(), test['label_id'].values.tolist()
x_dev, y_dev = dev['cleaned_comment'].values.tolist(), dev['label_id'].values.tolist()

In [ ]:
# Vectorize text reviews to numbers
vec = CountVectorizer()
x = vec.fit_transform(x_train)#.toarray()
x_test = vec.transform(x_test)#.toarray()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
model.score(x_test, y_test)

0.8432857142857143

In [ ]:
model.predict(vec.transform(['خیلی سرد بود اصلا دوست نداشتیم']))

array([1])

In [ ]:
model.predict(vec.transform(['عالی و خوشمزه بود']))

array([0])